In [50]:
!pip install rdflib sparqlwrapper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [51]:
geodata = {
  "type"     : "FeatureCollection",
  "name"     : "joined",
  "crs"      : { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },
  "features" : [{
      "type": "Feature", "properties": { 
          "name": "Basilica di Santo Stefano Rotondo", 
          "id": "relation/1576101", 
          "image": "http://commons.wikimedia.org/wiki/Special:FilePath/Celio%20-%20santo%20Stefano%20Rotondo%20-%20interno%20in%20restauro%2001533-4.JPG", 
          "place": "http://www.wikidata.org/entity/Q919456", 
          "placeLabel": "basilica di Santo Stefano Rotondo al Celio", 
          "gnd": "4302056-2", 
          "iccd": 15527 }
      } , {
      "type": "Feature", "properties": { 
          "name": "San Francesco di Paola ai Monti",
          "id": "way/157407150", 
          "image": "http://commons.wikimedia.org/wiki/Special:FilePath/Monti%20-%20S.%20Francesco%20di%20Paola.JPG",
          "place": "http://www.wikidata.org/entity/Q1581615", 
          "placeLabel": "chiesa di San Francesco di Paola", 
          "gnd": "7563217-2", 
          "iccd": None }
      } , {
      "type": "Feature", "properties": { 
          "name": "Chiesa di San Luigi dei Francesi", 
          "id": "way/52335646", 
          "image": "http://commons.wikimedia.org/wiki/Special:FilePath/San%20Luigi%20dei%20Francesi%20Church.jpg", 
          "place": "http://www.wikidata.org/entity/Q638283", 
          "placeLabel": "chiesa di San Luigi dei Francesi", 
          "gnd": "4199094-8", 
          "iccd": None }
      }]
}

In [52]:
gnd_ids = [ props['gnd'] for props in (feat['properties'] for feat in geodata['features'] ) ]
print(gnd_ids)
gndstr =  ' '.join(f'"{w}"' for w in gnd_ids)
gndstr

['4302056-2', '7563217-2', '4199094-8']


'"4302056-2" "7563217-2" "4199094-8"'

In [53]:
from SPARQLWrapper import SPARQLWrapper, JSON
from tabulate import tabulate

prefixes = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX rcp: <http://data.biblhertz.it/romacommunispatria/term/>
PREFIX crmsoc: <http://www.cidoc-crm.org/cidoc-crm/CRMsoc/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX  dct:<http://purl.org/dc/terms/>
PREFIX  bibo:<http://purl.org/ontology/bibo/>
PREFIX  frbr:<http://purl.org/vocab/frbr/core#>

"""

sparql = SPARQLWrapper('http://data.biblhertz.it/sparql/')

q = prefixes + """
SELECT DISTINCT ?x ?link WHERE {
  VALUES ?gnd { """ +  gndstr + """ }
  BIND ( IRI(CONCAT("https://d-nb.info/gnd/", ?gnd)) AS ?link)
  ?x (owl:sameAs|^owl:sameAs)* ?link
}
"""

sparql.setQuery(q)
sparql.setReturnFormat(JSON)
try:
    ret = sparql.queryAndConvert()
    links = {}
    for r in ret["results"]["bindings"]:
      if r['link']['value'] != r['x']['value']:
        links.setdefault(r['link']['value'], []).append(r['x']['value'])
    print(tabulate(links, headers=links.keys()))
except Exception as e:
    print(e)

https://d-nb.info/gnd/4302056-2                                           https://d-nb.info/gnd/7563217-2                                           https://d-nb.info/gnd/4199094-8
------------------------------------------------------------------------  ------------------------------------------------------------------------  ------------------------------------------------------------------------
http://data.biblhertz.it/builtwork/lvpa/87dd8d495162c03f2ba4dad0ae3a2645  http://data.biblhertz.it/builtwork/lvpa/b7e76dbc38f4c11e4ddd14300dcfdb78  http://data.biblhertz.it/builtwork/lvpa/31251f31311faa61d67bb6a40bcce55b
http://data.biblhertz.it/builtwork/zuccaro/100                            http://data.biblhertz.it/builtwork/zuccaro/71                             http://data.biblhertz.it/builtwork/zuccaro/912
http://data.biblhertz.it/romacommunispatria/builtwork/NB53                http://data.biblhertz.it/romacommunispatria/builtwork/NB07                http://data.biblhertz.it/romacommun

In [54]:
q = prefixes + """
SELECT DISTINCT ?gnd ?otherNames ?rite ?assignee ?architect WHERE {
  VALUES ?gnd { """ +  gndstr + """ }
  BIND ( IRI(CONCAT("https://d-nb.info/gnd/", ?gnd)) AS ?link)
  ?x (owl:sameAs|^owl:sameAs)* ?link
  . OPTIONAL { ?x rcp:had_or_has_Christian_denomination/skos:prefLabel ?rite }
  . OPTIONAL { ?x ^crmsoc:P6_to/crmsoc:P7_to/skos:prefLabel ?assignee }
  . OPTIONAL { ?x skos:altLabel ?otherNames }
  . OPTIONAL { ?x crm:108i_was_produced_by/crm:P01i_is_domain_of [ crm:P02_has_range/skos:preferredLabel ?architect ; crm:P14.1_in_the_role_of 	
<http://data.biblhertz.it/role/architekt> ] }
}
"""
sparql.setQuery(q)
try:
    ret = sparql.queryAndConvert()
    print(ret["results"]["bindings"])
except Exception as e:
    print(e)

[{'gnd': {'type': 'literal', 'value': '4302056-2'}}, {'gnd': {'type': 'literal', 'value': '4302056-2'}, 'otherNames': {'xml:lang': 'it', 'type': 'literal', 'value': 'Santo Stefano Rotondo al Celio, Santo Stefano in Girimonte, Santo Stefano in Querquetulano'}, 'rite': {'xml:lang': 'en', 'type': 'literal', 'value': 'Latin'}, 'assignee': {'xml:lang': 'it', 'type': 'literal', 'value': 'ungheresi'}}, {'gnd': {'type': 'literal', 'value': '7563217-2'}}, {'gnd': {'type': 'literal', 'value': '7563217-2'}, 'architect': {'type': 'literal', 'value': 'Luigi Barattone'}}, {'gnd': {'type': 'literal', 'value': '7563217-2'}, 'rite': {'xml:lang': 'en', 'type': 'literal', 'value': 'Latin'}, 'assignee': {'xml:lang': 'it', 'type': 'literal', 'value': 'calabresi'}}, {'gnd': {'type': 'literal', 'value': '4199094-8'}}, {'gnd': {'type': 'literal', 'value': '4199094-8'}, 'architect': {'type': 'literal', 'value': 'Antoine Derizet'}}, {'gnd': {'type': 'literal', 'value': '4199094-8'}, 'architect': {'type': 'liter

In [55]:
attrs = [ 'assignee', 'otherNames' , 'architect', 'rite' ]
for r in ret["results"]["bindings"]:
  gnd = r['gnd']['value']
  props = geodata['features'][gnd_ids.index(gnd)]['properties']
  for a in attrs:
    if a in r :
      a_ = props.setdefault(a,[])
      if r[a]['value'] not in a_ :
        a_.append(r[a]['value'])
geodata

{'type': 'FeatureCollection',
 'name': 'joined',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'type': 'Feature',
   'properties': {'name': 'Basilica di Santo Stefano Rotondo',
    'id': 'relation/1576101',
    'image': 'http://commons.wikimedia.org/wiki/Special:FilePath/Celio%20-%20santo%20Stefano%20Rotondo%20-%20interno%20in%20restauro%2001533-4.JPG',
    'place': 'http://www.wikidata.org/entity/Q919456',
    'placeLabel': 'basilica di Santo Stefano Rotondo al Celio',
    'gnd': '4302056-2',
    'iccd': 15527,
    'assignee': ['ungheresi'],
    'otherNames': ['Santo Stefano Rotondo al Celio, Santo Stefano in Girimonte, Santo Stefano in Querquetulano'],
    'rite': ['Latin']}},
  {'type': 'Feature',
   'properties': {'name': 'San Francesco di Paola ai Monti',
    'id': 'way/157407150',
    'image': 'http://commons.wikimedia.org/wiki/Special:FilePath/Monti%20-%20S.%20Francesco%20di%20Paola.JPG',
    'place': 'http://www.wikidata.org

In [56]:
import requests

endpoint = 'https://gn.biblhertz.it/fotothek/api/normdatei/architectur'
for gnd in gnd_ids:
  resp = requests.get(endpoint + '/gnd' + gnd)
  data = resp.json()
  print(data)

[]
[{'id': '08050553', 'title': 'San Francesco di Paola dei Calabresi'}]
[]


In [63]:

sparql = SPARQLWrapper('https://lod.b3kat.de/sparql')

q = prefixes + """
SELECT DISTINCt * WHERE { 
?s dct:subject ?y
 ; frbr:exemplar/frbr:owner <http://lod.b3kat.de/bib/DE-Y2>
FILTER (STR(?y)="http://d-nb.info/gnd/7563217-2")
}
"""

sparql.setQuery(q)
sparql.setReturnFormat(JSON)

ret = sparql.queryAndConvert()
ret

{'head': {'vars': ['s', 'y']},
 'results': {'bindings': [{'s': {'type': 'uri',
     'value': 'http://lod.b3kat.de/title/BV038093268'},
    'y': {'type': 'uri', 'value': 'http://d-nb.info/gnd/7563217-2'}},
   {'s': {'type': 'uri', 'value': 'http://lod.b3kat.de/title/BV038217608'},
    'y': {'type': 'uri', 'value': 'http://d-nb.info/gnd/7563217-2'}},
   {'s': {'type': 'uri', 'value': 'http://lod.b3kat.de/title/BV040894002'},
    'y': {'type': 'uri', 'value': 'http://d-nb.info/gnd/7563217-2'}},
   {'s': {'type': 'uri', 'value': 'http://lod.b3kat.de/title/BV041699931'},
    'y': {'type': 'uri', 'value': 'http://d-nb.info/gnd/7563217-2'}},
   {'s': {'type': 'uri', 'value': 'http://lod.b3kat.de/title/BV042507866'},
    'y': {'type': 'uri', 'value': 'http://d-nb.info/gnd/7563217-2'}}]}}